In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM mapping_categories WHERE categorie='nitrite'

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
NO3,1340,NITRATES (EN NO3),Nitrates (en NO3),None,nitrite
NO3_NO2,6374,NITRATES/50 + NITRITES/3,Nitrates/50 + Nitrites/3,None,nitrite
NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),None,nitrite
NO3,1340,NITRATES (EN NO3),Nitrates (en NO3),14797-55-8,nitrite
NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),14797-65-0,nitrite


In [3]:
%%sql --save int__resultats_nitrites_udi_dernier
WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'nitrite'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

split_nitrites AS (
    SELECT
        last_pvl.cdreseau,
        last_pvl.categorie,
        COUNT(DISTINCT last_pvl.cdparametresiseeaux) AS nb_parametres,
        MAX(last_pvl.datetimeprel) AS dernier_prel_datetime,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n03,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n02,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_no3_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3_no2
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
    GROUP BY
        last_pvl.cdreseau,
        last_pvl.categorie
)

SELECT
    split_nitrites.cdreseau,
    split_nitrites.categorie,
    'dernier_prel' AS periode,
    split_nitrites.dernier_prel_datetime,
    split_nitrites.nb_parametres,
    CASE
        WHEN
            DATE_DIFF(
                'day',
                LEAST(
                    split_nitrites.dernier_prel_datetime_n03,
                    split_nitrites.dernier_prel_datetime_n02,
                    split_nitrites.dernier_prel_datetime_no3_no2
                ),
                GREATEST(
                    split_nitrites.dernier_prel_datetime_n03,
                    split_nitrites.dernier_prel_datetime_n02,
                    split_nitrites.dernier_prel_datetime_no3_no2
                )
            ) >= 30
            OR split_nitrites.valtraduite_no2 IS NULL
            OR split_nitrites.valtraduite_no3 IS NULL
            THEN 'donnee_manquante'
        WHEN
            split_nitrites.valtraduite_no3_no2 IS NULL
            AND split_nitrites.valtraduite_no2 IS NOT NULL
            AND split_nitrites.valtraduite_no3 IS NOT NULL
            AND split_nitrites.valtraduite_no3 < 50
            AND split_nitrites.valtraduite_no2 < 0.5
            AND split_nitrites.valtraduite_no3 / 50
            + split_nitrites.valtraduite_no2 / 3
            < 1
            THEN 'conforme'
        WHEN
            split_nitrites.nb_parametres = 3
            AND split_nitrites.valtraduite_no3 < 50
            AND split_nitrites.valtraduite_no2 < 0.5
            AND split_nitrites.valtraduite_no3_no2 < 1
            THEN 'conforme'
        WHEN
            split_nitrites.valtraduite_no3 >= 50
            OR split_nitrites.valtraduite_no2 >= 0.5
            OR split_nitrites.valtraduite_no3_no2 >= 1
            THEN 'non_conforme'
        /*        WHEN
            split_nitrites.nb_parametres != 3
            AND (
                split_nitrites.valtraduite_no3 < 50
                OR split_nitrites.valtraduite_no2 < 0.5
                OR split_nitrites.valtraduite_no3_no2 < 1
            )
            THEN 'non_quantifie'*/
        ELSE 'error'
    END AS resultat
FROM
    split_nitrites


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
038001345,nitrite,dernier_prel,2024-12-03 09:10:00,3,conforme
038001363,nitrite,dernier_prel,2024-10-17 10:15:00,3,donnee_manquante
038001404,nitrite,dernier_prel,2024-09-13 08:05:00,3,conforme
038001576,nitrite,dernier_prel,2024-10-15 10:10:00,3,donnee_manquante
038001625,nitrite,dernier_prel,2024-12-17 10:25:00,3,conforme
038001899,nitrite,dernier_prel,2024-11-13 07:35:00,3,conforme
038001960,nitrite,dernier_prel,2024-10-14 11:25:00,3,conforme
038002035,nitrite,dernier_prel,2024-10-09 08:55:00,3,conforme
038002151,nitrite,dernier_prel,2024-11-18 10:25:00,3,conforme
038002176,nitrite,dernier_prel,2024-10-07 06:50:00,3,conforme


In [4]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    * 
FROM 
    int__resultats_nitrites_udi_dernier
WHERE 
    resultat='error'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat


In [5]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    resultat, MIN( cdreseau) min_cdreseau, MAX(cdreseau) AS max_cdreseau
FROM 
    int__resultats_nitrites_udi_dernier
GROUP BY 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,min_cdreseau,max_cdreseau
non_conforme,002000060,095000307
conforme,001000003,976003487
donnee_manquante,001000251,976003554


# Check 

In [8]:
%%sql
WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'nitrite'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
)
    SELECT
        last_pvl.cdreseau,
        last_pvl.categorie,
        COUNT(DISTINCT last_pvl.cdparametresiseeaux) AS nb_parametres,
        MAX(last_pvl.datetimeprel) AS dernier_prel_datetime,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n03,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n02,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_no3_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3_no2
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
        AND cdreseau IN( '002000060','095000346','001000003','976003554','088002296','073002059','973000028','001000260','001000268','001000293')
    GROUP BY
        last_pvl.cdreseau,
        last_pvl.categorie
    ORDER BY 
        cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,nb_parametres,dernier_prel_datetime,dernier_prel_datetime_n03,dernier_prel_datetime_n02,dernier_prel_datetime_no3_no2,valtraduite_no3,valtraduite_no2,valtraduite_no3_no2
001000003,nitrite,3,2024-12-26 09:08:00,2024-12-26 09:08:00,2024-12-26 09:08:00,2024-12-26 09:08:00,13.000,0.000,0.260
001000260,nitrite,3,2024-12-06 12:02:00,2024-12-06 12:02:00,2024-12-06 12:02:00,2024-12-06 12:02:00,2.400,0.000,0.050
001000268,nitrite,3,2024-11-28 08:25:00,2024-11-28 08:25:00,2024-11-28 08:25:00,2024-11-28 08:25:00,2.600,0.000,0.050
001000293,nitrite,3,2024-10-10 10:20:00,2024-10-10 10:20:00,2024-10-10 10:20:00,2024-10-10 10:20:00,4.000,0.000,0.080
002000060,nitrite,3,2024-10-18 08:50:00,2024-10-18 08:50:00,2024-10-18 08:50:00,2024-10-18 08:31:00,59.300,0.000,0.000
073002059,nitrite,1,2024-10-23 09:28:00,None,2024-10-23 09:28:00,None,None,0.000,None
088002296,nitrite,1,2024-09-27 12:00:00,None,2024-09-27 12:00:00,None,None,0.000,None
095000346,nitrite,3,2024-12-17 08:55:00,2024-12-17 08:55:00,2024-07-01 09:20:00,2024-07-01 09:20:00,51.000,0.000,1.000
973000028,nitrite,2,2024-12-02 11:30:00,2024-12-02 11:30:00,2024-12-02 11:30:00,None,0.000,0.000,None
976003554,nitrite,2,2024-11-12 10:00:00,2024-11-12 10:00:00,2024-09-10 09:20:00,None,0.000,0.000,None


In [7]:
%%sql --with int__resultats_nitrites_udi_dernier 
 SELECT 
   *
 FROM
   int__resultats_nitrites_udi_dernier
WHERE
    cdreseau IN( '002000060','095000346','001000003','976003554','088002296','073002059','973000028','001000260','001000268','001000293')
ORDER BY 
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
001000003,nitrite,dernier_prel,2024-12-26 09:08:00,3,conforme
001000260,nitrite,dernier_prel,2024-12-06 12:02:00,3,conforme
001000268,nitrite,dernier_prel,2024-11-28 08:25:00,3,conforme
001000293,nitrite,dernier_prel,2024-10-10 10:20:00,3,conforme
002000060,nitrite,dernier_prel,2024-10-18 08:50:00,3,non_conforme
073002059,nitrite,dernier_prel,2024-10-23 09:28:00,1,donnee_manquante
088002296,nitrite,dernier_prel,2024-09-27 12:00:00,1,donnee_manquante
095000346,nitrite,dernier_prel,2024-12-17 08:55:00,3,donnee_manquante
973000028,nitrite,dernier_prel,2024-12-02 11:30:00,2,conforme
976003554,nitrite,dernier_prel,2024-11-12 10:00:00,2,donnee_manquante
